<a href="https://colab.research.google.com/github/jacobgreen4477/Construction-Equipment-Oil-Condition-Classification-AI-Competition/blob/main/ETRI_v1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> title : 제 4회 ETRI 휴먼이해 인공지능 논문경진대회 <br>
> author : hjy <br>

In [6]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
from tqdm.auto import tqdm
import pandas as pd
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

warnings.filterwarnings('ignore')

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# pandas 옵션
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%0.4f' % x)

In [9]:
def plot_rssi_by_subject_topN(df, TOPN):
    """
    subject_id별로 가장 자주 등장한 상위 N개 BSSID에 대해 RSSI 시계열 시각화
    - RSSI -70 이하 신호는 제외 (None 처리)
    - 선 투명도 50%
    """
    grouped = df.groupby("subject_id")

    for subject, group in grouped:
        timestamps = []
        bssid_counter = Counter()

        # 모든 BSSID 등장 횟수 카운트
        for _, row in group.iterrows():
            bssid_counter.update(row['bssid'])

        # TOP N BSSID 추출
        target_bssids = [b for b, _ in bssid_counter.most_common(TOPN)]
        time_series = {bssid: [] for bssid in target_bssids}

        # 시간별 RSSI 수집 (단, -60 이하는 제거)
        for _, row in group.iterrows():
            timestamps.append(pd.to_datetime(row['timestamp']))
            row_bssid = row['bssid']
            row_rssi = row['rssi']

            for bssid in target_bssids:
                if bssid in row_bssid:
                    idx = row_bssid.index(bssid)
                    rssi_value = row_rssi[idx]
                    if rssi_value > -60:
                        time_series[bssid].append(rssi_value)
                    else:
                        time_series[bssid].append(None)  # 약한 신호는 제외
                else:
                    time_series[bssid].append(None)  # 없는 BSSID

        # 시각화
        plt.figure(figsize=(16, 7))
        for bssid in target_bssids:
            plt.plot(timestamps, time_series[bssid], label=bssid, marker='o', alpha=0.5)

        plt.title(f"[{subject}] 시간별 RSSI 변화 (TOP {TOPN}, -60 이상만)", fontsize=14)
        plt.xlabel("시간")
        plt.ylabel("RSSI (dBm)")
        plt.xticks(rotation=45)
        plt.legend(title="BSSID", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [10]:
def filter_strong_rssi(df, threshold=-60):
    """
    각 row의 RSSI 값 중 threshold 이하인 항목 제거

    Parameters:
    - df: DataFrame with 'bssid' and 'rssi' as lists
    - threshold: int, RSSI 값 기준 (기본: -60)

    Returns:
    - 필터링된 DataFrame (in-place 수정 아님)
    """
    filtered_df = df.copy()

    def filter_row(row):
        bssids = row['bssid']
        rssis = row['rssi']
        # RSSI > threshold 조건 만족하는 항목만 추출
        filtered = [(b, r) for b, r in zip(bssids, rssis) if r > threshold]
        if filtered:
            new_bssids, new_rssis = zip(*filtered)
            return pd.Series({'bssid': list(new_bssids), 'rssi': list(new_rssis)})
        else:
            return pd.Series({'bssid': [], 'rssi': []})

    filtered_df[['bssid', 'rssi']] = filtered_df.apply(filter_row, axis=1)
    return filtered_df

In [11]:
def daily_wifi_features_by_user(df):
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    features = []

    # subject_id + lifelog_date 기준 그룹화
    grouped = df.groupby(['subject_id', 'lifelog_date'])

    for (subject_id, date), group in grouped:
        scan_count = len(group)
        bssid_flat = sum(group['bssid'], [])  # flatten
        rssi_flat = sum(group['rssi'], [])    # flatten

        unique_bssid_count = len(set(bssid_flat))
        avg_rssi = sum(rssi_flat) / len(rssi_flat) if rssi_flat else None
        max_rssi = max(rssi_flat) if rssi_flat else None
        min_rssi = min(rssi_flat) if rssi_flat else None
        strong_rssi_ratio = sum(1 for r in rssi_flat if r > -60) / len(rssi_flat) if rssi_flat else 0
        empty_scan_count = sum(1 for b in group['bssid'] if len(b) == 0)

        # 가장 많이 탐지된 BSSID
        bssid_counter = Counter(bssid_flat)
        top_bssid, top_bssid_count = bssid_counter.most_common(1)[0] if bssid_counter else (None, 0)

        first_time = group['timestamp'].min()
        last_time = group['timestamp'].max()
        hour_span = (last_time - first_time).total_seconds() / 60  # 분 단위

        features.append({
            'subject_id': subject_id,
            'lifelog_date': date,
            'scan_count': scan_count,
            'unique_bssid_count': unique_bssid_count,
            'avg_rssi': avg_rssi,
            'max_rssi': max_rssi,
            'min_rssi': min_rssi,
            'strong_signal_ratio': strong_rssi_ratio,
            'empty_scan_count': empty_scan_count,
            'top_bssid': top_bssid,
            'top_bssid_count': top_bssid_count,
            #'time_first_seen': first_time,
            # 'time_last_seen': last_time,
            'hour_span_minutes': hour_span
        })

    return pd.DataFrame(features)

### 데이터 읽기

In [12]:
path = '/content/drive/MyDrive/data/ch2025_data_items/'

# 1
ch2025_mACStatus = pd.read_parquet(path+'ch2025_mACStatus.parquet')
ch2025_mActivity = pd.read_parquet(path+'ch2025_mActivity.parquet')
ch2025_mAmbience = pd.read_parquet(path+'ch2025_mAmbience.parquet')
ch2025_mBle = pd.read_parquet(path+'ch2025_mBle.parquet')
ch2025_mGps = pd.read_parquet(path+'ch2025_mGps.parquet')
ch2025_mLight = pd.read_parquet(path+'ch2025_mLight.parquet')
ch2025_mScreenStatus = pd.read_parquet(path+'ch2025_mScreenStatus.parquet')
ch2025_mUsageStats = pd.read_parquet(path+'ch2025_mUsageStats.parquet')
ch2025_mWifi = pd.read_parquet(path+'ch2025_mWifi.parquet')
ch2025_wHr = pd.read_parquet(path+'ch2025_wHr.parquet')
ch2025_wLight = pd.read_parquet(path+'ch2025_wLight.parquet')
ch2025_wPedo = pd.read_parquet(path+'ch2025_wPedo.parquet')

# 2
train = pd.read_csv('/content/drive/MyDrive/data/ch2025_metrics_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/ch2025_submission_sample.csv')

### mWifi
- Wifi devices around individual subject.

In [13]:
def extract_wifi_info(row):
    wifi_data = row['m_wifi']
    bssids = [item['bssid'] for item in wifi_data]
    rssis = [item['rssi'] for item in wifi_data]
    return pd.Series({'bssid': bssids, 'rssi': rssis})

ch2025_mWifi[['bssid', 'rssi']] = ch2025_mWifi.apply(extract_wifi_info, axis=1)
ch2025_mWifi['lifelog_date'] = ch2025_mWifi['timestamp'].astype(str).str[:10]
ch2025_mWifi = ch2025_mWifi.drop(columns=['m_wifi'])
ch2025_mWifi.head(1)

,subject_id,timestamp,bssid,rssi,lifelog_date
0,id01,2024-06-26 12:03:00,"[a0:0f:37:9a:5d:8b, a0:0f:37:9a:5d:8c, a0:0f:37:9a:5d:8d, a0:0f:37:9a:5d:8e, a0:0f:37:9a:5d:8f, a0:0f:37:96:56:ef, 88:36:6c:86:75:84, a0:0f:37:96:56:ee, a0:0f:37:96:56:ed, 86:25:19:b5:b2:a5, a0:0f:37:96:56:ec, 1e:39:29:8e:fb:e9, 52:c2:e8:c7:9b:e4, a0:0f:37:96:56:eb, 12:e3:c7:09:20:34, 58:86:94:4a:08:b8, 90:9f:33:28:d0:2e, 00:26:66:bc:4e:18, f6:0a:f4:43:4b:ba, 10:e3:c7:09:20:35, 10:e3:c7:09:20:34, 1c:39:29:48:04:92, 12:e3:c7:07:9d:df, 86:25:19:c3:44:07, a0:0f:37:9a:37:2f, a0:0f:37:9a:37:2e, a0:0f:37:9a:37:2d, 0a:09:b4:74:05:ec, a0:0f:37:9a:37:2c, a0:0f:37:9a:37:2b, 0a:09:b4:74:05:eb, c0:25:2f:d8:c1:a6, 16:7f:67:bb:fa:f8, 3c:f3:92:ff:00:01, 06:09:b4:74:05:ec, 06:09:b4:74:05:eb, 12:e3:c7:0a:74:d1, 88:36:6c:a9:6f:8e, 02:e3:c7:09:20:34, 00:09:b4:74:05:eb, 00:09:b4:74:05:ec, 00:1d:93:93:cf:fe, 8e:e2:ac:a5:9d:15]","[-78, -78, -78, -78, -78, -58, -72, -58, -58, -61, -58, -71, -82, -58, -88, -82, -78, -85, -45, -63, -89, -82, -83, -84, -76, -76, -76, -72, -76, -76, -59, -82, -79, -82, -72, -59, -78, -63, -88, -60, -72, -19, -72]",2024-06-26


In [ ]:
# wifi 약신호 제거
# ch2025_mWifi = filter_strong_rssi(ch2025_mWifi, threshold=-60)
# ch2025_mWifi.head(1)

In [14]:
ch2025_mWifi_daily = daily_wifi_features_by_user(ch2025_mWifi)
ch2025_mWifi_daily.head()

,subject_id,lifelog_date,scan_count,unique_bssid_count,avg_rssi,max_rssi,min_rssi,strong_signal_ratio,empty_scan_count,top_bssid,top_bssid_count,hour_span_minutes
0,id01,2024-06-26,69,393,-70.1964,-19,-91,0.2309,0,86:25:19:9f:9b:be,19,716.0000
1,id01,2024-06-27,126,357,-69.0629,-26,-92,0.2701,0,04:09:a5:3a:c8:6a,54,1430.0000
2,id01,2024-06-28,118,376,-69.0941,-26,-92,0.2594,0,04:09:a5:3a:c8:6a,47,1430.0000
3,id01,2024-06-29,134,258,-67.7897,-24,-91,0.3063,0,04:09:a5:3a:c8:6a,117,1420.0000
4,id01,2024-06-30,108,242,-68.2999,-23,-90,0.2946,0,04:09:a5:3a:c8:6a,70,1310.0000


### mAmbience
- Ambient sound identification labels and their respective probabilities.

In [15]:
# - mAmbience: Ambient sound identification labels and their respective probabilities.

def extract_labels_and_probs(row):
    items = row['m_ambience']
    labels = [item[0] for item in items]
    probs = [item[1] for item in items]
    return pd.Series({'labels': labels, 'prob': probs})

ch2025_mAmbience[['labels', 'prob']]  = ch2025_mAmbience.apply(extract_labels_and_probs, axis=1)
ch2025_mAmbience['lifelog_date'] = ch2025_mAmbience['timestamp'].astype(str).str[:10]
ch2025_mAmbience = ch2025_mAmbience.drop(columns=['m_ambience'])
ch2025_mAmbience.head(1)

,subject_id,timestamp,labels,prob,lifelog_date
0,id01,2024-06-26 13:00:10,"[Music, Vehicle, Motor vehicle (road), Outside, urban or manmade, Outside, rural or natural, Car, Speech, Inside, large room or hall, Truck, Sound effect]","[0.30902618, 0.081680894, 0.04035286, 0.037144363, 0.032663062, 0.03199804, 0.029806137, 0.01684492, 0.016206821, 0.01591479]",2024-06-26


In [16]:
ch2025_mAmbience.head(2)

,subject_id,timestamp,labels,prob,lifelog_date
0,id01,2024-06-26 13:00:10,"[Music, Vehicle, Motor vehicle (road), Outside, urban or manmade, Outside, rural or natural, Car, Speech, Inside, large room or hall, Truck, Sound effect]","[0.30902618, 0.081680894, 0.04035286, 0.037144363, 0.032663062, 0.03199804, 0.029806137, 0.01684492, 0.016206821, 0.01591479]",2024-06-26
1,id01,2024-06-26 13:02:10,"[Music, Vehicle, Bell, Hiss, Jingle bell, Chime, Car, Motor vehicle (road), Bicycle, Chink, clink]","[0.62307084, 0.021118319, 0.018510727, 0.013137147, 0.012887808, 0.012369333, 0.011385, 0.0107314605, 0.010630278, 0.010412726]",2024-06-26


In [20]:
import pandas as pd
from scipy.stats import entropy

def generate_derived_features(df,
                             calculate_mean=True,
                             calculate_max=True,
                             calculate_entropy=True,
                             calculate_presence=False,
                             target_label=None):

    # 데이터 파싱 개선 함수
    def parse_list(x):
        if isinstance(x, str):
            # 1. 대괄호 제거
            cleaned = x.strip('[]')
            # 2. 쉼표(,) 기준 분리 (공백 유무 관계없이 처리)
            items = [item.strip() for item in cleaned.split(',')]
            # 3. 숫자인 경우 float 변환, 레이블은 문자열 유지
            try:
                return list(map(float, items)) if '.' in cleaned else items
            except:
                return items
        return x

    # 데이터 전처리 (공통)
    df = df.assign(
        labels=df['labels'].apply(parse_list),
        prob=df['prob'].apply(parse_list)
    ).explode(['labels', 'prob'])

    # 타입 강제 변환 (추가 안전장치)
    df['prob'] = pd.to_numeric(df['prob'], errors='coerce')
    df = df.dropna(subset=['prob'])

    # 그룹화 기준
    group_keys = ['subject_id', 'lifelog_date']

    # 결과 저장 딕셔너리
    results = {}

    # 1. 평균 확률 계산
    if calculate_mean:
        mean_prob = df.groupby(group_keys + ['labels'])['prob'].mean().unstack().reset_index()
        results['mean_probability'] = mean_prob

    # 2. 최대 확률 레이블
    if calculate_max:
        max_prob = df.loc[df.groupby(group_keys)['prob'].idxmax()][group_keys + ['labels', 'prob']]
        results['max_prob_label'] = max_prob

    # 3. 엔트로피 계산
    if calculate_entropy:
        entropy_df = df.groupby(group_keys + ['labels'])['prob'].mean() \
                     .groupby(group_keys).apply(entropy).reset_index(name='entropy')
        results['label_entropy'] = entropy_df

    # 4. 특정 레이블 존재 여부
    if calculate_presence and target_label:
        presence_df = df.groupby(group_keys)['labels'] \
                      .apply(lambda x: (x == target_label).any().astype(int)) \
                      .reset_index(name=f'has_{target_label}')
        results['label_presence'] = presence_df

    return results

# 사용 예시
results = generate_derived_features(
    ch2025_mAmbience,
    calculate_mean=True,
    calculate_max=True,
    calculate_entropy=True,
    calculate_presence=True,
    target_label='Music'
)

results.head(1)

AttributeError: 'dict' object has no attribute 'head'

In [23]:
results['label_presence'].head()

,subject_id,lifelog_date,has_Music
0,id01,2024-06-26,1
1,id01,2024-06-27,1
2,id01,2024-06-28,1
3,id01,2024-06-29,1
4,id01,2024-06-30,1


### 학습 & 테스트 데이터

In [ ]:
train2 = train.merge(ch2025_mWifi_daily,on=['subject_id','lifelog_date'],how='left')
train2.head()

,subject_id,sleep_date,lifelog_date,Q1,Q2,Q3,S1,S2,S3,scan_count,unique_bssid_count,avg_rssi,max_rssi,min_rssi,strong_signal_ratio,empty_scan_count,top_bssid,top_bssid_count,hour_span_minutes
0,id01,2024-06-27,2024-06-26,0,0,0,0,0,1,69.0000,393.0000,-70.1964,-19.0000,-91.0000,0.2309,0.0000,86:25:19:9f:9b:be,19.0000,716.0000
1,id01,2024-06-28,2024-06-27,0,0,0,0,1,1,126.0000,357.0000,-69.0629,-26.0000,-92.0000,0.2701,0.0000,04:09:a5:3a:c8:6a,54.0000,1430.0000
2,id01,2024-06-29,2024-06-28,1,0,0,1,1,1,118.0000,376.0000,-69.0941,-26.0000,-92.0000,0.2594,0.0000,04:09:a5:3a:c8:6a,47.0000,1430.0000
3,id01,2024-06-30,2024-06-29,1,0,1,2,0,0,134.0000,258.0000,-67.7897,-24.0000,-91.0000,0.3063,0.0000,04:09:a5:3a:c8:6a,117.0000,1420.0000
4,id01,2024-07-01,2024-06-30,0,1,1,1,1,1,108.0000,242.0000,-68.2999,-23.0000,-90.0000,0.2946,0.0000,04:09:a5:3a:c8:6a,70.0000,1310.0000


In [ ]:
test2 = test.merge(ch2025_mWifi_daily,on=['subject_id','lifelog_date'],how='left')
test2.head()

,subject_id,sleep_date,lifelog_date,Q1,Q2,Q3,S1,S2,S3,scan_count,unique_bssid_count,avg_rssi,max_rssi,min_rssi,strong_signal_ratio,empty_scan_count,top_bssid,top_bssid_count,hour_span_minutes
0,id01,2024-07-31,2024-07-30,0,0,0,0,0,0,115.0000,910.0000,-73.8566,-29.0000,-93.0000,0.1451,0.0000,86:25:19:9f:9b:be,42.0000,1400.0000
1,id01,2024-08-01,2024-07-31,0,0,0,0,0,0,135.0000,492.0000,-69.2624,-33.0000,-93.0000,0.2579,0.0000,04:09:a5:3a:c8:6a,48.0000,1430.0000
2,id01,2024-08-02,2024-08-01,0,0,0,0,0,0,124.0000,397.0000,-72.1787,-33.0000,-92.0000,0.2053,0.0000,86:25:19:9f:9b:be,49.0000,1428.0000
3,id01,2024-08-03,2024-08-02,0,0,0,0,0,0,132.0000,366.0000,-69.7404,-27.0000,-93.0000,0.2188,0.0000,04:09:a5:3a:c8:6a,53.0000,1430.0000
4,id01,2024-08-04,2024-08-03,0,0,0,0,0,0,107.0000,312.0000,-69.6403,-21.0000,-93.0000,0.2214,0.0000,04:09:a5:3a:c8:6a,77.0000,1420.0000
